In [2]:
from pynq import Overlay
import time
ol = Overlay('matrix.bit')

In [3]:
#ol?

In [4]:
dma = ol.axi_dma_0

In [5]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [6]:
import numpy as np
import random 

In [7]:
random.seed(2)
data_A = np.random.uniform(low=0, high=10, size=(8, 8)).astype(float)
data_B = np.random.uniform(low=0, high=10, size=(8, 8)).astype(float)

In [8]:
data_A, data_B

(array([[2.84070953, 8.00958085, 5.44342667, 8.0298136 , 0.43705535,
         3.69399812, 4.24278048, 2.12296445],
        [8.13549247, 5.32455498, 1.9975075 , 2.54243682, 6.40635884,
         0.02820824, 8.48502296, 9.44281387],
        [2.12644218, 1.68500586, 9.77631359, 3.70504845, 1.52767446,
         8.71868709, 2.85118974, 9.29298741],
        [0.03218976, 1.19002627, 9.91426427, 8.07671621, 6.24551761,
         7.70067587, 4.31554854, 3.90684105],
        [1.38482497, 8.21868496, 5.91619398, 1.91123408, 9.43425496,
         4.52682652, 2.57405053, 4.86857646],
        [9.61889794, 8.01419503, 4.20002485, 5.3257203 , 3.1710348 ,
         9.17633084, 0.68947276, 9.27548267],
        [1.52858565, 5.52588208, 1.27548022, 9.9424612 , 1.13533356,
         0.02628193, 4.78819774, 3.45136359],
        [9.11195094, 1.00287662, 5.01186736, 8.11797677, 8.82097486,
         4.85396878, 2.13674754, 4.51392867]]),
 array([[2.68931035, 0.40421186, 1.61613584, 6.06820029, 0.15870874,
         

In [9]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            for k in range(8):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [10]:
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

0.015656471252441406


In [11]:
from pynq import allocate

In [12]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [13]:
data_input

array([2.84070953, 8.00958085, 5.44342667, 8.0298136 , 0.43705535,
       3.69399812, 4.24278048, 2.12296445, 8.13549247, 5.32455498,
       1.9975075 , 2.54243682, 6.40635884, 0.02820824, 8.48502296,
       9.44281387, 2.12644218, 1.68500586, 9.77631359, 3.70504845,
       1.52767446, 8.71868709, 2.85118974, 9.29298741, 0.03218976,
       1.19002627, 9.91426427, 8.07671621, 6.24551761, 7.70067587,
       4.31554854, 3.90684105, 1.38482497, 8.21868496, 5.91619398,
       1.91123408, 9.43425496, 4.52682652, 2.57405053, 4.86857646,
       9.61889794, 8.01419503, 4.20002485, 5.3257203 , 3.1710348 ,
       9.17633084, 0.68947276, 9.27548267, 1.52858565, 5.52588208,
       1.27548022, 9.9424612 , 1.13533356, 0.02628193, 4.78819774,
       3.45136359, 9.11195094, 1.00287662, 5.01186736, 8.11797677,
       8.82097486, 4.85396878, 2.13674754, 4.51392867, 2.68931035,
       0.40421186, 1.61613584, 6.06820029, 0.15870874, 1.02097666,
       7.41215982, 8.55789922, 1.42906595, 6.93176592, 6.06703

In [14]:
input_buffer = allocate(128, np.float32)
output_buffer = allocate((8,8), np.float32)

In [15]:
np.copyto(input_buffer, data_input)

In [16]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.002740621566772461


In [17]:
output_buffer

PynqBuffer([[136.44678 , 222.24678 , 157.00839 , 142.35541 , 203.36285 ,
             124.12585 , 166.5234  , 161.86047 ],
            [180.79091 , 207.1365  , 175.05487 , 261.78915 , 179.3591  ,
             175.65234 , 245.0537  , 258.02786 ],
            [168.42395 , 211.5405  , 123.213295, 173.02695 , 212.37418 ,
             177.36311 , 256.58612 , 173.89633 ],
            [216.34215 , 258.7161  , 167.58731 , 155.62343 , 207.9961  ,
             218.96169 , 246.43001 , 118.366165],
            [184.20404 , 218.59778 , 172.2034  , 182.28319 , 191.51787 ,
             184.26141 , 253.02661 , 164.52246 ],
            [191.75471 , 223.70142 , 197.79126 , 230.84006 , 225.62976 ,
             179.04211 , 297.91455 , 264.10223 ],
            [111.21345 , 185.94182 , 165.0733  , 120.86467 , 146.68921 ,
             101.32382 ,  98.49532 , 134.20688 ],
            [213.50313 , 209.69044 , 194.79758 , 186.92572 , 142.90263 ,
             199.74608 , 266.06696 , 173.28793 ]], dtype=float32)

In [18]:
diff = np.dot(data_A, data_B) - output_buffer

In [19]:
rms_error = np.sqrt(np.mean(diff**2))

In [20]:
diff, rms_error

(PynqBuffer([[-3.24405298e-06, -1.32379498e-06,  9.34636026e-06,
              -4.44546237e-06, -1.79156736e-05,  3.74116583e-06,
               1.27199090e-06,  6.79349844e-06],
             [ 1.11112774e-05, -6.00890024e-06,  6.78029889e-06,
              -2.67095970e-06,  2.04185506e-05,  1.46761174e-05,
              -3.77588714e-06, -9.34042521e-06],
             [-6.98413223e-06, -8.32748512e-06, -6.33965811e-07,
              -2.02753799e-06,  1.46892954e-05, -2.81361918e-06,
              -1.88030646e-06, -3.06999050e-06],
             [-5.46125366e-06,  3.98740195e-06, -1.01809322e-05,
               2.67707151e-06, -4.97251534e-07, -2.72692535e-06,
              -6.80895380e-06,  2.49721231e-06],
             [ 2.33247940e-06, -2.20193833e-05, -9.56321063e-06,
               4.83719191e-06, -1.44208391e-05,  1.97661300e-05,
              -6.95201859e-06, -7.96294722e-06],
             [-1.01535162e-05, -2.14211829e-05, -2.68393217e-06,
              -6.86412400e-06, -3.697599